DATA PIPELINE 2021 - CAPSTONE PROJECT

1. Load Data for 2021  final_df2021

2021 

In [1]:
import os
import pandas as pd
from tqdm import tqdm  # Progress bar


# Define the correct folder path
folder_path = "/home/fm/Desktop/Data2021"

# Get all CSV files in the folder
csv_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.csv')]

# Create an empty list to store DataFrames
dfs = []

# Progress bar for loading files
print("Loading CSV files...")
for file in tqdm(csv_files, desc="Files Processed", unit="file"):
    chunk_list = []
    
    # Use tqdm to show chunk loading progress
    for chunk in tqdm(pd.read_csv(file, chunksize=50000), desc=f"Processing {os.path.basename(file)}", unit="chunk"):
        chunk_list.append(chunk)
    
    # Concatenate chunks for the current file
    df = pd.concat(chunk_list, ignore_index=True)
    dfs.append(df)


# Concatenate all files into a single DataFrame
final_df2021 = pd.concat(dfs, ignore_index=True)

# Clear list for next year's dataframes   
dfs.clear()

# Display the first few rows
print(final_df2021.head())

# Show DataFrame shape
print("Shape of the DataFrame:", final_df2021.shape)


Loading CSV files...


Processing 2021_05_Maig_BicingNou_ESTACIONS.csv: 91chunk [00:01, 57.03chunk/s]
Processing 2021_04_Abril_BicingNou_ESTACIONS.csv: 88chunk [00:01, 63.44chunk/s]
Processing 2021_09_Setembre_BicingNou_ESTACIONS.csv: 88chunk [00:01, 75.40chunk/s]
Processing 2021_03_Març_BicingNou_ESTACIONS.csv: 91chunk [00:01, 76.03chunk/s]
Processing 2021_12_Desembre_BicingNou_ESTACIONS.csv: 88chunk [00:01, 71.53chunk/s]
Processing 2021_02_Febrer_BicingNou_ESTACIONS.csv: 81chunk [00:01, 78.55chunk/s]
Processing 2021_01_Gener_BicingNou_ESTACIONS.csv: 91chunk [00:01, 75.12chunk/s]
Processing 2021_06_Juny_BicingNou_ESTACIONS.csv: 87chunk [00:01, 74.87chunk/s]
Processing 2021_07_Juliol_BicingNou_ESTACIONS.csv: 89chunk [00:01, 73.07chunk/s]
Processing 2021_08_Agost_BicingNou_ESTACIONS.csv: 91chunk [00:01, 74.37chunk/s]
Processing 2021_10_Octubre_BicingNou_ESTACIONS.csv: 91chunk [00:01, 76.84chunk/s]
Processing 2021_11_Novembre_BicingNou_ESTACIONS.csv: 87chunk [00:01, 70.79chunk/s]
Files Processed: 100%|████████

   station_id  num_bikes_available  num_bikes_available_types.mechanical  \
0           1                   27                                    27   
1           2                   26                                    25   
2           3                   26                                    26   
3           4                    7                                     4   
4           5                    4                                     4   

   num_bikes_available_types.ebike  num_docks_available  last_reported  \
0                                0                   19   1.619820e+09   
1                                1                    0   1.619820e+09   
2                                0                    0   1.619820e+09   
3                                3                   13   1.619820e+09   
4                                0                   35   1.619820e+09   

   is_charging_station      status  is_installed  is_renting  is_returning  \
0                 Tr

2. Checking all labels to see if we have missing values 

2021

In [2]:
# Check for missing values
missing_values = final_df2021.isnull().sum()

# Show columns with missing values only
missing_values = missing_values[missing_values > 0]

# Display missing values
if missing_values.empty:
    print("No missing values found in the dataset.")
else:
    print("Missing values per column:\n")
    print(missing_values)

missing_percentage = (final_df2021.isnull().sum() / len(final_df2021)) * 100
missing_percentage = missing_percentage[missing_percentage > 0]
print("Percentage of missing values per column:\n", missing_percentage)

# Clear missing values list for the next year

missing_values = []

# Print structure of the DataFrame
print(final_df2021.head())

Missing values per column:

traffic    52014085
dtype: int64
Percentage of missing values per column:
 traffic    98.420109
dtype: float64
   station_id  num_bikes_available  num_bikes_available_types.mechanical  \
0           1                   27                                    27   
1           2                   26                                    25   
2           3                   26                                    26   
3           4                    7                                     4   
4           5                    4                                     4   

   num_bikes_available_types.ebike  num_docks_available  last_reported  \
0                                0                   19   1.619820e+09   
1                                1                    0   1.619820e+09   
2                                0                    0   1.619820e+09   
3                                3                   13   1.619820e+09   
4                                0

3. Removing columbns traffic, V1 and last_updated and also the missing values from the other labels for year 2021

2021

In [3]:
# Define columns to drop completely


columns_to_drop = []
columns_to_drop = ["last_updated","traffic",] # There is no "V1",in 2021 and 2022 data
final_df_cleaned2021 = final_df2021.drop(columns=columns_to_drop)


# Drop rows with any remaining missing values

final_df_cleaned2021 = final_df_cleaned2021.dropna()


# Display the shape after cleaning

print("Shape of DataFrame 2021 after cleaning:", final_df_cleaned2021.shape)


# Check if any missing values remain

print("Remaining missing values 2021:\n", final_df_cleaned2021.isnull().sum())


# Display the first few rows
#print(final_df_cleaned.head())


Shape of DataFrame 2021 after cleaning: (52849042, 12)
Remaining missing values 2021:
 station_id                              0
num_bikes_available                     0
num_bikes_available_types.mechanical    0
num_bikes_available_types.ebike         0
num_docks_available                     0
last_reported                           0
is_charging_station                     0
status                                  0
is_installed                            0
is_renting                              0
is_returning                            0
ttl                                     0
dtype: int64


4. Checking if the numbers are integers or not 

2021

In [4]:

import numpy as np
import pandas as pd

# List of numeric columns to check
numeric_columns = [
    "station_id",
    "num_bikes_available",
    "num_bikes_available_types.mechanical",
    "num_bikes_available_types.ebike",
    "num_docks_available",
    "is_installed",
    "is_renting",
    "is_returning",
    "ttl"
]

# Vectorized integer check using NumPy for speed
integer_check = []
integer_check = final_df_cleaned2021[numeric_columns].apply(lambda col: np.isclose(col, np.round(col)))

# Count non-integer values per column
non_integer_counts = (integer_check == False).sum()

# Print results
if non_integer_counts.sum() == 0:
    print("✅ All numeric values are integers for 2021.")
else:
    print("⚠️ Some numeric values are NOT integers:")
    print(non_integer_counts[non_integer_counts > 0])

✅ All numeric values are integers for 2021.


4.1 Removing negative values

2021

In [5]:
# Detect negative values and count them per column
negative_counts = (final_df_cleaned2021[numeric_columns] < 0).sum()
columns_with_negatives = negative_counts[negative_counts > 0]

# Display columns with negative values before replacing
if not columns_with_negatives.empty:
    print("⚠️ Columns with negative values detected (before replacement):")
    print(columns_with_negatives)
    
    # Replace negative values with 0
    final_df_cleaned2021[numeric_columns] = final_df_cleaned2021[numeric_columns].clip(lower=0)
    print("\n✅ Negative values have been replaced with 0.")
else:
    print("✅ No negative values detected. No replacement needed.")





✅ No negative values detected. No replacement needed.


5. Converting the date 

2021

In [6]:


# Convert 'last_reported' from Unix timestamp to datetime
final_df_cleaned2021["last_reported"] = pd.to_datetime(final_df_cleaned2021["last_reported"], unit='s')

# Sort DataFrame in ascending order to keep the earliest record per hour per station
final_df_cleaned2021.sort_values(by=["station_id", "last_reported"], ascending=True, inplace=True)

# Extract date and hour for grouping
final_df_cleaned2021["date"] = final_df_cleaned2021["last_reported"].dt.date
final_df_cleaned2021["hour"] = final_df_cleaned2021["last_reported"].dt.hour

# Keep only the earliest record per hour per day per station
final_df_hourly2021 = final_df_cleaned2021.drop_duplicates(subset=["station_id", "date", "hour"], keep="first").copy()

# Add additional columns
final_df_hourly2021["weekday"] = final_df_hourly2021["last_reported"].dt.day_name()
final_df_hourly2021["weekday_num"] = final_df_hourly2021["last_reported"].dt.weekday + 1
final_df_hourly2021["month"] = final_df_hourly2021["last_reported"].dt.month_name()
final_df_hourly2021["month_num"] = final_df_hourly2021["last_reported"].dt.month

# Reset the index
final_df_hourly2021.reset_index(drop=True, inplace=True)

# Display preview
print(final_df_hourly2021[["station_id", "last_reported", "date", "hour", "weekday", "weekday_num", "month", "month_num"]].head())


display(final_df_hourly2021.head(1))



   station_id       last_reported        date  hour   weekday  weekday_num  \
0           1 2020-12-31 22:56:46  2020-12-31    22  Thursday            4   
1           1 2020-12-31 23:01:13  2020-12-31    23  Thursday            4   
2           1 2021-01-01 00:03:27  2021-01-01     0    Friday            5   
3           1 2021-01-01 01:00:59  2021-01-01     1    Friday            5   
4           1 2021-01-01 02:02:42  2021-01-01     2    Friday            5   

      month  month_num  
0  December         12  
1  December         12  
2   January          1  
3   January          1  
4   January          1  


,station_id,num_bikes_available,num_bikes_available_types.mechanical,num_bikes_available_types.ebike,num_docks_available,last_reported,is_charging_station,status,is_installed,is_renting,is_returning,ttl,date,hour,weekday,weekday_num,month,month_num
0,1,43,43,0,3,2020-12-31 22:56:46,True,IN_SERVICE,1,1,1,22,2020-12-31,22,Thursday,4,December,12


6. Adding capacity from the Excel file 

2021

In [7]:


# Define the correct Excel file path
excel_path = "/home/fm/Desktop/Data2023/Informacio_Estacions_Bicing_2025.xlsx"

# Load the Excel file
station_info_df = pd.read_excel(excel_path)

# Ensure 'station_id' and 'capacity' are integers
station_info_df["station_id"] = station_info_df["station_id"].astype(int)
station_info_df["capacity"] = station_info_df["capacity"].astype(int)

# Check and remove existing 'capacity' columns if they exist to avoid merge conflict
cols_to_remove = [col for col in ['capacity', 'capacity_x', 'capacity_y'] if col in final_df_hourly2021.columns]
if cols_to_remove:
    final_df_hourly2021 = final_df_hourly2021.drop(columns=cols_to_remove)

# Merge 'capacity' into the main DataFrame using 'station_id' as the key
final_df_hourly2021 = final_df_hourly2021.merge(
    station_info_df[["station_id", "capacity"]],
    on="station_id",
    how="left"
)

# Display a preview after merging


# Check if 'capacity' is correctly merged and contains integers
print("Data types after merging:\n", final_df_hourly2021.dtypes)
display(final_df_hourly2021.head(1))

Data types after merging:
 station_id                                       int64
num_bikes_available                              int64
num_bikes_available_types.mechanical             int64
num_bikes_available_types.ebike                  int64
num_docks_available                              int64
last_reported                           datetime64[ns]
is_charging_station                               bool
status                                          object
is_installed                                     int64
is_renting                                       int64
is_returning                                     int64
ttl                                              int64
date                                            object
hour                                             int32
weekday                                         object
weekday_num                                      int32
month                                           object
month_num                             

,station_id,num_bikes_available,num_bikes_available_types.mechanical,num_bikes_available_types.ebike,num_docks_available,last_reported,is_charging_station,status,is_installed,is_renting,is_returning,ttl,date,hour,weekday,weekday_num,month,month_num,capacity
0,1,43,43,0,3,2020-12-31 22:56:46,True,IN_SERVICE,1,1,1,22,2020-12-31,22,Thursday,4,December,12,46


7. Calculating 'percentage_docks_available'

2021

In [8]:


# Calculate 'percentage_docks_available'
final_df_hourly2021["percentage_docks_available"] = (1 - (final_df_hourly2021["num_bikes_available"] / final_df_hourly2021["capacity"])) * 100

# Handle potential division by zero cases (if capacity is 0) - Avoid FutureWarning
final_df_hourly2021["percentage_docks_available"] = final_df_hourly2021["percentage_docks_available"].fillna(0)

# Display a preview
print(final_df_hourly2021[["station_id", "num_bikes_available", "capacity", "percentage_docks_available"]].head()) 

display(final_df_hourly2021.head(1))


   station_id  num_bikes_available  capacity  percentage_docks_available
0           1                   43        46                    6.521739
1           1                   43        46                    6.521739
2           1                   41        46                   10.869565
3           1                   41        46                   10.869565
4           1                   41        46                   10.869565


,station_id,num_bikes_available,num_bikes_available_types.mechanical,num_bikes_available_types.ebike,num_docks_available,last_reported,is_charging_station,status,is_installed,is_renting,is_returning,ttl,date,hour,weekday,weekday_num,month,month_num,capacity,percentage_docks_available
0,1,43,43,0,3,2020-12-31 22:56:46,True,IN_SERVICE,1,1,1,22,2020-12-31,22,Thursday,4,December,12,46,6.521739


7.1 There are a few stations wchich are missing capacity in Informacio_Estacions_Bicing_2025.xlsx, this I identified and added manually their capacity to Informacio_Estacions_Bicing_2025.xlsx



2021 check

In [9]:


# Identify stations with missing capacity
missing_capacity_stations = final_df_hourly2021[final_df_hourly2021["capacity"].isna()]
print("Stations with missing capacity:")
print(missing_capacity_stations[["station_id", "num_bikes_available", "num_docks_available", "capacity"]])

# Count unique station IDs with NaN in capacity
num_stations_with_nan_capacity = final_df_hourly2021.loc[
    final_df_hourly2021["capacity"].isna(), "station_id"
].nunique()

print(f"Number of unique stations with NaN in capacity: {num_stations_with_nan_capacity}")

# Identify stations with NaN in capacity
nan_capacity_df = final_df_hourly2021[final_df_hourly2021["capacity"].isna()]

# Number of unique stations with NaN capacity
num_unique_nan_stations = nan_capacity_df["station_id"].nunique()
print(f"Number of unique stations with NaN in capacity: {num_unique_nan_stations}\n")

# Group by station_id, count rows, and find the maximum sum per station
station_summary = nan_capacity_df.groupby("station_id").agg(
    num_rows=("station_id", "count"),
    max_total_available=("num_bikes_available", lambda x: (x + nan_capacity_df.loc[x.index, "num_docks_available"]).max())
).reset_index()

print("Stations with NaN capacity summary:")
print(station_summary)


Stations with missing capacity:
Empty DataFrame
Columns: [station_id, num_bikes_available, num_docks_available, capacity]
Index: []
Number of unique stations with NaN in capacity: 0
Number of unique stations with NaN in capacity: 0

Stations with NaN capacity summary:
Empty DataFrame
Columns: [station_id, num_rows, max_total_available]
Index: []


8. There are stations where "percentage_docks_available" is more then 100% , report it and if more than 100 percent make to be equal to 100 percent

2021

In [10]:


# Filter rows where 'percentage_docks_available' is greater than 100%
over_100_stations = final_df_hourly2021[final_df_hourly2021["percentage_docks_available"] > 100]

# Get the list of affected station IDs
over_100_station_ids = over_100_stations["station_id"].unique()

# Display the station IDs
print("🔴 Stations where percentage_docks_available > 100% BEFORE:")
print(over_100_station_ids)

# Replace values greater than 100% with 100%
final_df_hourly2021["percentage_docks_available"] = final_df_hourly2021["percentage_docks_available"].clip(upper=100)

# Filter rows where 'percentage_docks_available' is greater than 100%
over_100_stations = final_df_hourly2021[final_df_hourly2021["percentage_docks_available"] > 100]

# Get the list of affected station IDs
over_100_station_ids = over_100_stations["station_id"].unique()

# Display the station IDs
print("🔴 Stations where percentage_docks_available > 100% AFTER:")
print(over_100_station_ids)

# Display the DataFrame in Jupyter Notebook
display(final_df_hourly2021.head(1))




🔴 Stations where percentage_docks_available > 100% BEFORE:
[]
🔴 Stations where percentage_docks_available > 100% AFTER:
[]


,station_id,num_bikes_available,num_bikes_available_types.mechanical,num_bikes_available_types.ebike,num_docks_available,last_reported,is_charging_station,status,is_installed,is_renting,is_returning,ttl,date,hour,weekday,weekday_num,month,month_num,capacity,percentage_docks_available
0,1,43,43,0,3,2020-12-31 22:56:46,True,IN_SERVICE,1,1,1,22,2020-12-31,22,Thursday,4,December,12,46,6.521739


9. Open Meteo API and Python Code - Extraction

In [11]:
# INSTALL THE PIPS IF YOU DO NOT HAVE THEM (I installed them in my virtual environment in Linux and do not need them in my code)
# pip install openmeteo-requests
# pip install requests-cache retry-requests numpy pandas


import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
	"latitude": 41.3888,
	"longitude": 2.159,
	"start_date": "2020-01-01",
	"end_date": "2024-12-31",
	"hourly": ["temperature_2m", "rain", "apparent_temperature", "relative_humidity_2m", "wind_speed_10m", "is_day", "sunshine_duration"],
	"timezone": "Europe/Berlin"
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()}{response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

							# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
hourly_rain = hourly.Variables(1).ValuesAsNumpy()
hourly_apparent_temperature = hourly.Variables(2).ValuesAsNumpy()
hourly_relative_humidity_2m = hourly.Variables(3).ValuesAsNumpy()
hourly_wind_speed_10m = hourly.Variables(4).ValuesAsNumpy()
hourly_is_day = hourly.Variables(5).ValuesAsNumpy()
hourly_sunshine_duration = hourly.Variables(6).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}

hourly_data["temperature_2m"] = hourly_temperature_2m
hourly_data["rain"] = hourly_rain
hourly_data["apparent_temperature"] = hourly_apparent_temperature
hourly_data["relative_humidity_2m"] = hourly_relative_humidity_2m
hourly_data["wind_speed_10m"] = hourly_wind_speed_10m
hourly_data["sunshine_duration"] = hourly_sunshine_duration

hourly_dataframe = pd.DataFrame(data = hourly_data)
print(hourly_dataframe)




Coordinates 41.37082290649414°N 2.068965435028076°E
Elevation 44.0 m asl
Timezone b'Europe/Berlin'b'GMT+1'
Timezone difference to GMT+0 3600 s
                           date  temperature_2m  rain  apparent_temperature  \
0     2019-12-31 23:00:00+00:00            4.64   0.0              1.536496   
1     2020-01-01 00:00:00+00:00            4.24   0.0              0.874873   
2     2020-01-01 01:00:00+00:00            3.69   0.0              0.518975   
3     2020-01-01 02:00:00+00:00            3.29   0.0              0.066232   
4     2020-01-01 03:00:00+00:00            2.79   0.0             -0.455314   
...                         ...             ...   ...                   ...   
43843 2024-12-31 18:00:00+00:00            8.89   0.0              8.038103   
43844 2024-12-31 19:00:00+00:00            8.24   0.0              6.649469   
43845 2024-12-31 20:00:00+00:00            7.64   0.0              5.839178   
43846 2024-12-31 21:00:00+00:00            7.54   0.0              

9.1 Checking the data types for the merger with the meteo data (just for 2023 is enough)

In [13]:
import pandas as pd

# Generate report of columns and their data types
columns_report = final_df_hourly2021.dtypes.reset_index()
columns_report.columns = ["Column Label", "Data Type"]

print("📋 Column Labels and Data Types Report:")
print(columns_report)



📋 Column Labels and Data Types Report:
                            Column Label       Data Type
0                             station_id           int64
1                    num_bikes_available           int64
2   num_bikes_available_types.mechanical           int64
3        num_bikes_available_types.ebike           int64
4                    num_docks_available           int64
5                          last_reported  datetime64[ns]
6                    is_charging_station            bool
7                                 status          object
8                           is_installed           int64
9                             is_renting           int64
10                          is_returning           int64
11                                   ttl           int64
12                                  date          object
13                                  hour           int32
14                               weekday          object
15                           weekday_num         

In [14]:
# Generate and display column names and data types
print("📌 Labels and Data Types in the Weather DataFrame:")
print(hourly_dataframe.dtypes.reset_index().rename(columns={"index": "Label", 0: "Data Type"}))


📌 Labels and Data Types in the Weather DataFrame:
                  Label            Data Type
0                  date  datetime64[ns, UTC]
1        temperature_2m              float32
2                  rain              float32
3  apparent_temperature              float32
4  relative_humidity_2m              float32
5        wind_speed_10m              float32
6     sunshine_duration              float32


9.2. Splitting the date and the hour in the meteo file in order to have the hour seperate for the merge

In [15]:
# Correctly generate the datetime range explicitly from hourly.Time()
hourly_dataframe["date"] = pd.date_range(
    start=pd.to_datetime(hourly.Time(), unit="s", utc=True),
    periods=len(hourly_temperature_2m),
    freq=pd.Timedelta(seconds=hourly.Interval())
).tz_convert("Europe/Berlin")

# Add weather variables to the dataframe
hourly_dataframe["temperature_2m"] = hourly_temperature_2m
hourly_dataframe["rain"] = hourly_rain
hourly_dataframe["apparent_temperature"] = hourly_apparent_temperature
hourly_dataframe["relative_humidity_2m"] = hourly_relative_humidity_2m
hourly_dataframe["wind_speed_10m"] = hourly_wind_speed_10m
hourly_dataframe["sunshine_duration"] = hourly_sunshine_duration

# Extract new date and hour columns explicitly
hourly_dataframe["datenew"] = hourly_dataframe["date"].dt.strftime('%Y-%m-%d')
hourly_dataframe["hour"] = hourly_dataframe["date"].dt.hour.astype('int32')

# Verify corrected extraction
print("✅ Corrected extraction sample:")
print(hourly_dataframe[['date', 'datenew', 'hour']].head(10))

# Clearly confirm hours
print("\n✅ Unique hours after timezone correction:")
print(sorted(hourly_dataframe['hour'].unique()))


display(hourly_dataframe.head(1))



✅ Corrected extraction sample:
                       date     datenew  hour
0 2020-01-01 00:00:00+01:00  2020-01-01     0
1 2020-01-01 01:00:00+01:00  2020-01-01     1
2 2020-01-01 02:00:00+01:00  2020-01-01     2
3 2020-01-01 03:00:00+01:00  2020-01-01     3
4 2020-01-01 04:00:00+01:00  2020-01-01     4
5 2020-01-01 05:00:00+01:00  2020-01-01     5
6 2020-01-01 06:00:00+01:00  2020-01-01     6
7 2020-01-01 07:00:00+01:00  2020-01-01     7
8 2020-01-01 08:00:00+01:00  2020-01-01     8
9 2020-01-01 09:00:00+01:00  2020-01-01     9

✅ Unique hours after timezone correction:
[np.int32(0), np.int32(1), np.int32(2), np.int32(3), np.int32(4), np.int32(5), np.int32(6), np.int32(7), np.int32(8), np.int32(9), np.int32(10), np.int32(11), np.int32(12), np.int32(13), np.int32(14), np.int32(15), np.int32(16), np.int32(17), np.int32(18), np.int32(19), np.int32(20), np.int32(21), np.int32(22), np.int32(23)]


,date,temperature_2m,rain,apparent_temperature,relative_humidity_2m,wind_speed_10m,sunshine_duration,datenew,hour
0,2020-01-01 00:00:00+01:00,4.64,0.0,1.536496,85.641792,9.504273,0.0,2020-01-01,0


10. Add the weather information to the dataframes per year

READY: 2021 dataframe: final_df_hourly2021

In [16]:


# Ensure 'date' and 'hour' columns are correctly formatted for merging
final_df_hourly2021['date'] = pd.to_datetime(final_df_hourly2021['date']).dt.strftime('%Y-%m-%d')
final_df_hourly2021['hour'] = final_df_hourly2021['hour'].astype('int32')

# Ensure 'datenew' in hourly_dataframe is also string and 'hour' is integer
hourly_dataframe['datenew'] = hourly_dataframe['datenew'].astype(str)
hourly_dataframe['hour'] = hourly_dataframe['hour'].astype('int32')

# Identify overlapping columns (excluding merge keys)
overlapping_columns = set(final_df_hourly2021.columns) & set(hourly_dataframe.columns) - {'date', 'hour', 'datenew'}
print("Overlapping columns before merge:", overlapping_columns)

# Drop duplicate columns from final_df_hourly before merging
final_df_hourly2021.drop(columns=overlapping_columns, inplace=True, errors='ignore')

# Rename columns in hourly_dataframe to avoid conflicts
hourly_dataframe = hourly_dataframe.rename(columns={
    'temperature_2m': 'temperature',
    'rain': 'raining',
    'apparent_temperature': 'feels_like_weather',
    'relative_humidity_2m': 'humidity',
    'wind_speed_10m': 'wind_speed',
    'sunshine_duration': 'sunshine'
})

# Merge the relevant columns from hourly_dataframe
final_df_hourly2021 = final_df_hourly2021.merge(
    hourly_dataframe[['datenew', 'hour', 'temperature', 'raining', 'feels_like_weather',
                      'humidity', 'wind_speed', 'sunshine']],
    left_on=['date', 'hour'],
    right_on=['datenew', 'hour'],
    how='left'
)

# Drop redundant 'datenew' column after merge
final_df_hourly2021.drop(columns=['datenew'], inplace=True)

# Verify the merged data
#print("\n✅ Data after merging weather information:")
#print(final_df_hourly2023.head(1))

# Confirm the successful merge and data types
print("\n✅ Data types after merging:")
print(final_df_hourly2021.dtypes)

display(final_df_hourly2021.head(1))


Overlapping columns before merge: set()

✅ Data types after merging:
station_id                                       int64
num_bikes_available                              int64
num_bikes_available_types.mechanical             int64
num_bikes_available_types.ebike                  int64
num_docks_available                              int64
last_reported                           datetime64[ns]
is_charging_station                               bool
status                                          object
is_installed                                     int64
is_renting                                       int64
is_returning                                     int64
ttl                                              int64
date                                            object
hour                                             int32
weekday                                         object
weekday_num                                      int32
month                                           obj

,station_id,num_bikes_available,num_bikes_available_types.mechanical,num_bikes_available_types.ebike,num_docks_available,last_reported,is_charging_station,status,is_installed,is_renting,...,month,month_num,capacity,percentage_docks_available,temperature,raining,feels_like_weather,humidity,wind_speed,sunshine
0,1,43,43,0,3,2020-12-31 22:56:46,True,IN_SERVICE,1,1,...,December,12,46,6.521739,7.99,0.0,5.641069,70.196846,4.896529,0.0


11. Download in CSV

In [17]:
import os
from tqdm import tqdm

# Define the file name and save location
output_file = "final_df_hourly2021.csv"
output_path = os.path.abspath(output_file)  # Get full absolute path

# Define chunk size (adjust depending on your system memory)
chunk_size = 100_000  

# Open file in write mode
with open(output_file, "w", encoding="utf-8", newline='') as file:
    # Write the header only once
    final_df_hourly2021.iloc[:0].to_csv(file, index=False)

    # Write the CSV in chunks with a progress bar
    with tqdm(total=len(final_df_hourly2021), desc="Saving CSV", unit="rows") as pbar:
        for i in range(0, len(final_df_hourly2021), chunk_size):
            final_df_hourly2021.iloc[i:i+chunk_size].to_csv(file, index=False, header=False, mode='a')
            pbar.update(min(chunk_size, len(final_df_hourly2021) - i))  # Avoid over-updating

# Print confirmation and file path
print(f"\n✅ CSV file saved at: {output_path}")


Saving CSV: 100%|██████████| 4407036/4407036 [00:20<00:00, 213892.47rows/s]


✅ CSV file saved at: /home/fm/Desktop/final_df_hourly2021.csv
